In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
from typing import List, Dict, Any
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import uuid

def perform_search(query: str, num_results: int = 1) -> List[str]:
    """
    Perform a web search using googlesearch-python.
    
    Args:
        query: The search query
        num_results: Number of results to return
        
    Returns:
        List of URLs
    """
    try:
        # Import the search function from the googlesearch-python package
        from googlesearch import search
        
        # Add delay between requests to avoid being blocked
        # Longer sleep_interval reduces chances of getting blocked
        urls = list(search(
            query,
            # num=num_results, 
            # sleep_interval=2,  # 5 second delay between requests
            # timeout=30  # 30 second timeout for each request
        ))
        
        return urls
    except Exception as e:
        print(f"Error with Google search: {e}")
        # Return empty list if search fails
        return []

def extract_text_from_url(url: str, timeout: int = 15) -> Dict[str, str]:
    """
    Extract text from a URL with improved reliability and retry logic.
    
    Args:
        url: The URL to extract text from
        timeout: Timeout in seconds
        
    Returns:
        Dict with url, title, and text
    """
    # Define different user agents to rotate through
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Safari/605.1.15',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:95.0) Gecko/20100101 Firefox/95.0'
    ]
    
    max_retries = 3
    retry_count = 0
    
    while retry_count < max_retries:
        try:
            # Use a random user agent
            headers = {
                'User-Agent': random.choice(user_agents),
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                'Accept-Language': 'en-US,en;q=0.5',
                'Accept-Encoding': 'gzip, deflate',
                'DNT': '1',
                'Connection': 'keep-alive',
                'Upgrade-Insecure-Requests': '1'
            }
            
            response = requests.get(url, headers=headers, timeout=timeout)
            response.raise_for_status()
            
            # Check content type to ensure it's text/html
            content_type = response.headers.get('Content-Type', '').lower()
            if 'text/html' not in content_type and 'application/xhtml+xml' not in content_type:
                # Skip non-HTML content
                return {"url": url, "title": "Non-HTML Content", "text": ""}
            
            # Parse with BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Get the title
            title = soup.title.string if soup.title else "No title"
            
            # Remove scripts, styles, and hidden elements
            for element in soup(['script', 'style', 'header', 'footer', 'nav']):
                element.decompose()
            
            # Extract text with better spacing
            for br in soup.find_all('br'):
                br.replace_with('\n')
            
            # Replace paragraph and header tags with newlines
            for tag in soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'li']):
                tag.append(soup.new_string('\n\n'))
            
            text = soup.get_text(separator=' ')
            
            # Clean up the text
            lines = [line.strip() for line in text.split('\n')]
            text = '\n'.join([line for line in lines if line])
            
            # Normalize whitespace
            text = ' '.join(text.split())
                
            return {
                "url": url,
                "title": title,
                "text": text
            }
            
        except requests.exceptions.RequestException as e:
            print(f"Request error extracting text from {url}: {e}")
            retry_count += 1
            time.sleep(random.uniform(2.0, 5.0))
            
        except Exception as e:
            print(f"Error extracting text from {url}: {e}")
            return {"url": url, "title": "Error", "text": ""}
    
    # If all retries failed
    return {"url": url, "title": "Failed after retries", "text": ""}

def extract_text_from_urls(urls: List[str]) -> List[Dict[str, str]]:
    """
    Extract text from multiple URLs in parallel with improved reliability.
    
    Args:
        urls: List of URLs to extract text from
        
    Returns:
        List of dictionaries with url, title, and text
    """
    results = []
    with ThreadPoolExecutor(max_workers=3) as executor:  # Reduced to avoid rate limiting
        future_to_url = {executor.submit(extract_text_from_url, url): url for url in tqdm(urls)}
        for future in as_completed(future_to_url):
            url = future_to_url[future]
            try:
                result = future.result()
                if result["text"]:  # Only append non-empty results
                    results.append(result)
            except Exception as e:
                print(f"Error processing {url}: {e}")
    
    return results

In [2]:
urls = perform_search("top restaurants in new york city")

In [3]:
results = extract_text_from_urls(urls[:5])

100%|██████████| 5/5 [00:00<00:00, 1969.71it/s]


In [ ]:
from vector_store import vector_store

texts = []
metadatas = []

for result in results:
    if result["text"]:
        # Store the text in the vector database
        texts.append(result["text"])
        
        # Create metadata
        metadata = {
            "url": result["url"],
            "title": result["title"],
            # "subtopic": subtopic,
            "source_id": str(uuid.uuid4())  # Unique ID for citation
        }
        metadatas.append(metadata)
        
        # Add to sources for citation
        sources.append(metadata)
        
        # Track source by subtopic (without summary)
        subtopic_sources[subtopic].append({
            "url": result["url"],
            "title": result["title"]
        })

# Add to vector store
if texts:
    vector_store.add_texts(texts, metadatas)

[{'url': 'https://guide.michelin.com/us/en/new-york-state/new-york/restaurants',
  'title': 'New York MICHELIN Restaurants - The MICHELIN Guide USA',
  'text': 'New York MICHELIN Restaurants - The MICHELIN Guide USA Restaurants Hotels Travel Guides Best-of Guides Magazine MICHELIN Guide Ceremony My Favorites Contact us Subscribe to newsletter Booking partnership with OpenTable Booking partnership with Resy USA - English - USD Restaurants All New Distinction Distinction Distinction 3 Stars Exceptional cuisine 5 2 Stars Excellent cooking 13 1 Star High quality cooking 54 Bib Gourmand Good quality, good value cooking 82 Selected Restaurants Good cooking 211 Green Star In MICHELIN Hotels Online booking Online booking Online booking 7 90 118 22 Delivery Days open Days open Days open Monday 234 Tuesday 281 Wednesday 312 Thursday 316 Friday 318 Saturday 318 Sunday 261 Hours open Dinner Lunch Cuisine Cuisine Cuisine Alsatian 1 American 37 American Contemporary 7 Argentinian 1 Asian 9 Asian Con

In [ ]:
from langchain_community.document_loaders import RecursiveUrlLoader
from bs4 import BeautifulSoup

